# 08 - Explainability and Fairness (V3)

- SHAP global and local explanations for finalist model
- Fairness slice metrics (precision/recall/FPR/FNR) by categorical groups
- Save plots and metrics CSV


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import shap
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import matplotlib.pyplot as plt

INP = Path('../v3_data/employee_promotion_features.csv')
ART = Path('../v3_artifacts'); ART.mkdir(exist_ok=True)

TARGET = 'Promotion_Eligible'
GROUP = 'Current_Position_Level'  # slice attribute if available

# Data
df = pd.read_csv(INP)
X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()

pre = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

model = GradientBoostingClassifier(random_state=42)
pipe = Pipeline([('pre', pre), ('model', model)])
pipe.fit(X_train, y_train)

# SHAP (tree explainer on underlying GB model after fit)
# Get transformed training features for summary plot
pre_fit = pipe.named_steps['pre']
Xtr = pre_fit.transform(X_train)
clf = pipe.named_steps['model']
try:
    explainer = shap.TreeExplainer(clf)
    shap_values = explainer.shap_values(Xtr)
    plt.figure()
    shap.summary_plot(shap_values, Xtr, show=False)
    plt.tight_layout()
    plt.savefig(ART / 'shap_summary.png', dpi=160)
    plt.close()
except Exception as e:
    print('SHAP failed with:', e)

# Fairness metrics by GROUP (if exists)
if GROUP in df.columns:
    proba = pipe.predict_proba(X_test)[:, 1]
    pred = (proba >= 0.5).astype(int)
    rows = []
    for g, idx in X_test.groupby(GROUP).groups.items():
        yt, pt = y_test.loc[idx], pred[idx]
        rows.append({
            GROUP: g,
            'precision': precision_score(yt, pt, zero_division=0),
            'recall': recall_score(yt, pt, zero_division=0)
        })
    pd.DataFrame(rows).to_csv(ART / 'fairness_slice_metrics.csv', index=False)
else:
    print(f'Slice attribute {GROUP} not found; skipping fairness slices')
